In [3]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import math
import re
import sys
from collections import Counter

from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import fdrcorrection

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils


In [ ]:
#Set where to write the files
writeDir = '/Users/friedman/Desktop/hypermutationProjectFinal/scripts/figure2/FIGURE2_PLOTTING_FILES/'

## Figure 2S(i)

In [ ]:
#Todo--just need to load the figure 2a dataframe

## Figure 2S(ii)

In [ ]:
def summarize_obs_vs_expected_by_gene_type(maf, relatedGenesDict, expectedDf, cancerTypeDict):
    impact341Genes = get_gene_and_cohort_list_utils.get_im3_genes()
    expectedDf = expectedDf[expectedDf['gene'].isin(impact341Genes)]
    maf = maf[maf['Hugo_Symbol'].isin(impact341Genes)]
    oncSnpMaf = maf[(maf['oncogenic'].notnull()) & (maf['Variant_Type'] == 'SNP')]
    mutCounts = dict(maf['Tumor_Sample_Barcode'].value_counts())
    listOfDicts = []
    cntr = 0
    for case, cancerType in cancerTypeDict.items():
        if case in set(maf['Tumor_Sample_Barcode']) and case in set(expectedDf['case']):
        
            if cntr%50 == 0: print cntr,
            caseMaf = oncSnpMaf[oncSnpMaf['Tumor_Sample_Barcode'] == case]
            caseExpectation = expectedDf[expectedDf['case'] == case]
            relatedGenes = relatedGenesDict[cancerType]

            relatedGenesMaf = caseMaf[caseMaf['Hugo_Symbol'].isin(relatedGenes)]
            notRelatedGenesMaf = caseMaf[~caseMaf['Hugo_Symbol'].isin(relatedGenes)]
            relatedGenesExpected = caseExpectation[caseExpectation['gene'].isin(relatedGenes)]
            notRelatedGenesExpected = caseExpectation[~caseExpectation['gene'].isin(relatedGenes)]

            listOfDicts.append({'cancerType': cancerType, 'Tumor_Sample_Barcode': case,
                                'nmut_IM341': mutCounts[case],
                'nRelatedObserved': relatedGenesMaf.shape[0],
                'nUnrelatedObserved': notRelatedGenesMaf.shape[0],
                'nRelatedExpected': mutCounts[case]*sum(relatedGenesExpected['oncogenicChance']),
                'nUnrelatedExpected': mutCounts[case]*sum(notRelatedGenesExpected['oncogenicChance'])
            })

            cntr += 1
        
    df = pd.DataFrame(listOfDicts)
    return df

In [ ]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
relatedGenesDict = get_gene_and_cohort_list_utils.get_related_genes_by_cancer_type()

#Load in data as needed
cancerTypeDict = dict(get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath = filePathDict['CANCER_TYPE_INFO']))
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
hypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalIds = get_gene_and_cohort_list_utils.get_all_normal_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
allImpactMutsMaf['hypermutationStatus'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x:
    'hypermutated' if x in hypermutantIds else 'normal' if x in normalIds else 'Intermediate')
hypermutantMaf = allImpactMutsMaf[allImpactMutsMaf['hypermutationStatus'] == 'hypermutated']

df = summarize_obs_vs_expected_by_gene_type(hypermutantMaf, relatedGenesDict, expectedDf, cancerTypeDict)

In [ ]:
writePath = os.path.join(writeDir, 'figure_2S_ii.tsv')
df.to_csv(writePath, index=False, sep='\t')

## Figure 2S(iii)

In [ ]:
df = pd.read_table('/Users/friedman/Desktop/hypermutationProjectFinal/tables/table5.tsv')
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
relatedGenesDict = get_gene_and_cohort_list_utils.get_related_genes_by_cancer_type()
df['related'] = df.apply(lambda row:
                'related' if row['CANCER_TYPE'] in relatedGenesDict and row['GENE'] in relatedGenesDict[row['CANCER_TYPE']]
                           else 'not-related', axis=1)
df['dndsIsSignificantHyper'] = df['DNDS_HYPER_SCORE'].apply(lambda x: 1 if x <.05 else 0)
df['cancerTypeGene'] = df.apply(lambda row: row['CANCER_TYPE'] + '_' + row['related'], axis=1)

In [ ]:
writePath = os.path.join(writeDir, 'figure_2S_iii.tsv')
df.to_csv(writePath, index=False, sep='\t')

## Figure 2S(iv)

In [ ]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath=filePathDict['CANCER_TYPE_INFO'])
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = filePathDict['CASE_TMB_AND_MSI_STATS'], msiScoreThresh=10)
apobecCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_APOBEC')
poleCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_10')
hypermutatedIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalIds = get_gene_and_cohort_list_utils.get_all_normal_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(normalIds)]

#summarize the data for different signature cohorts
#note more signature cohorts could be added here
listOfDfs = []
caseMapping = {'POLE': poleCases, 'APOBEC': apobecCases, 'MSI': msiCases}
for sigType, cases in caseMapping.items():
    print 'sampling', sigType
    hypermutatedMaf = allImpactMutsMaf[(allImpactMutsMaf['Tumor_Sample_Barcode'].isin(cases))]
    samples = sample_normal_maf(normalMaf, hypermutatedMaf, N=15)
    print 'summarizing'
    df = summarize_mutational_prevalence(hypermutatedMaf, samples)
    df['signature'] = sigType
    listOfDfs.append(df)


combinedDf = pd.concat(listOfDfs)
combinedDf['ratio'] = combinedDf['fracHyper']/combinedDf['fracNormal']
combinedDf['ddrGene'] = combinedDf['gene'].apply(lambda x: True if x in ['POLE', 'MLH1', 'MSH2', 'MSH6', 'PMS2'] else False)


In [ ]:
writePath = os.path.join(writeDir, 'figure_2S_iv.tsv')
df.to_csv(writePath, index=False, sep='\t')

## Figure 2S(v)

In [ ]:
#specfically look at pentanucleotide context in POLE cases and its relation to mutationa susceptibility
poleMafWithPentaContext = pd.read_table(filePathDict['PENTANUCLEOTIDE_CONTEXT_ANNOTATED_MAF'])
poleMafWithPentaContext['pentaChange'] = poleMafWithPentaContext.apply(lambda row: 
                                    mutationSigUtils.create_strand_specific_pentanucleotide_change(row['Ref_Tri.1'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)




## Figure 2S(vi)

In [ ]:
cancerTypes = ['Endometrial Cancer', 'Colorectal Cancer', 'Esophagogastric Cancer', 'Prostate Cancer']
listOfDfs = []
compsDone = []
for c1 in cancerTypes:
    for c2 in cancerTypes:
        comp = '/'.join(sorted((c1, c2)))
        if c1 != c2 and comp not in compsDone:
            cancerTypesMaf = allMsiCasesMaf[
                allMsiCasesMaf['cancerType'].isin([c1, c2]) & (allMsiCasesMaf['Tumor_Sample_Barcode'].isin(msiCases))]
            df = make_comparissons(cancerTypesMaf, mode='gene', cancerType1 = c1, cancerType2 = c2)
            df['comp'] = comp            
            compsDone.append(comp)
            listOfDfs.append(df)
combinedDf = pd.concat(listOfDfs)

In [ ]:
writePath = os.path.join(writeDir, 'figure_2S_v.tsv')
combinedDf.to_csv(writePath, index=False, sep='\t')


## Figure 2S(vii)

In [ ]:
poleCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_10')
poleEndometrialColorectalMaf = allImpactMutsMaf[
    (allImpactMutsMaf['cancerType'].isin(['Endometrial Cancer', 'Colorectal Cancer'])) & 
    (allImpactMutsMaf['Tumor_Sample_Barcode'].isin(poleCases))]
df = make_comparissons(poleEndometrialColorectalMaf, mode = 'gene', mutationType='pole')

In [ ]:
writePath = os.path.join(writeDir, 'figure_2S_i.tsv')
df.to_csv(writePath, index=False, sep='\t')

## Figure 2S(vii)

## Figure 2S(viii)

## SUPPLEMENTARY FIGURE: gene prevalene normal vs hyper by signature type

In [ ]:
def make_comparissons(maf, mode = 'gene', mutationType='msi',
        cancerType1 = 'Endometrial Cancer', cancerType2 = 'Colorectal Cancer'):
    tsgs = get_gene_and_cohort_list_utils.get_tsgs()
    oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()
    indelClassifications = ['Frame_Shift_Del', 'Frame_Shift_Ins']  #TODO actually only include MSI indels not just random indels
    truncatingClassifications = ['']
    
    if mutationType == 'msi':
        maf = maf[maf['Variant_Classification'].isin(indelClassifications)]
        maf = maf[maf['correctedAllele'].notnull()]
    elif mutationType == 'pole':
        maf = maf[maf['Variant_Classification'].isin(['Nonsense_Mutation'])] 
    else:
        pass
    
    if mode == 'gene':
        maf['allele'] = maf['Hugo_Symbol']
    else:
        maf['allele'] = maf['Hugo_Symbol'] + '_' + maf['HGVSp_Short']
    
    c1Maf = maf[maf['cancerType'] == cancerType1]
    c2Maf = maf[maf['cancerType'] == cancerType2]
    
    nC1 = 1.0*len(set(c1Maf['Tumor_Sample_Barcode']))
    nC2 = 1.0*len(set(c2Maf['Tumor_Sample_Barcode']))
    
    listOfDicts = []
    for allele in set(maf['allele']):
        allele = str(allele)
        aMafC1 = c1Maf[c1Maf['allele'] == allele]
        aMafC2 = c2Maf[c2Maf['allele'] == allele]
        
        gene = ''
        if mode == 'gene':
            gene = allele
        else:
            gene = allele.split('_')[0]
        
        
            
        geneType = 'tsg' if gene in tsgs else 'oncogene' if gene in oncogenes else None
        c1Count = len(set(aMafC1['Tumor_Sample_Barcode']))
        c2Count = len(set(aMafC2['Tumor_Sample_Barcode']))
        listOfDicts.append({'Allele': allele, 'GeneType': geneType, 'Gene': gene,
                            'N_C1': c1Count, 'N_C2': c2Count, 'total_C1': nC1, 'total_C2': nC2,
                            'c1_cancerType': cancerType1, 'c2_cancerType': cancerType2,
                           'perCase_c1': c1Count/nC1, 'perCase_c2': c2Count/nC2})
    
    df = pd.DataFrame(listOfDicts)    
    df['n_NotPresent_c1'] = df['N_C1'].apply(lambda x: nC1 - x)
    df['n_NotPresent_c2'] = df['N_C2'].apply(lambda x: nC2 - x)
    
    #get fisher's test results
    df['p_proportions_z_score'] = df.apply(lambda row: proportions_ztest(np.array([row['N_C1'], row['N_C2']]),
                                                                         np.array([nC1, nC2]))[1], axis=1)
    
    df = df[df['p_proportions_z_score'].notnull()] #remove null z scores, otherwise the qVal wont calculate
    fdrDict = dict(zip(df['Allele'], fdrcorrection(df['p_proportions_z_score'])[1]))
    df['qVal'] = df['Allele'].apply(lambda x: fdrDict[x])
    
    wntGenes = get_gene_and_cohort_list_utils.get_pathway_genes('WNT')
    pi3kGenes = get_gene_and_cohort_list_utils.get_pathway_genes('PI3K') | set(['INPPL1', 'JAK1']) #manually add these guys
    df['pathway'] = df['Gene'].apply(lambda x: 'WNT' if x in wntGenes else 'PI3K' if x in pi3kGenes else 'OTHER')
    return df

In [ ]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])

In [ ]:
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath=filePathDict['CANCER_TYPE_INFO'])
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = filePathDict['CASE_TMB_AND_MSI_STATS'], msiScoreThresh=10)
hypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])


In [ ]:
poleCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_10')
apobecCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_APOBEC')


In [ ]:
apobecMaf = allImpactMutsMaf[
    allImpactMutsMaf['Tumor_Sample_Barcode'].isin(apobecCases) &
    (allImpactMutsMaf['Tumor_Sample_Barcode'].isin(hypermutantIds))]

In [ ]:
#Counter(apobecMaf.drop_duplicates(subset=['Tumor_Sample_Barcode'])['cancerType'])

df = make_comparissons(apobecMaf, mode = 'gene', mutationType='',
        cancerType1 = 'Bladder Cancer', cancerType2 = 'Breast Cancer')



In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/apobecCompTest.tsv', index=False, sep='\t')

## Observed and expcted by related vs unrelated gene class

In [9]:
def summarize_obs_vs_expected_by_gene_type(maf, relatedGenesDict, expectedDf, cancerTypeDict):
    impact341Genes = get_gene_and_cohort_list_utils.get_im3_genes()
    expectedDf = expectedDf[expectedDf['gene'].isin(impact341Genes)]
    maf = maf[maf['Hugo_Symbol'].isin(impact341Genes)]
    oncSnpMaf = maf[(maf['oncogenic'].notnull()) & (maf['Variant_Type'] == 'SNP')]
    mutCounts = dict(maf['Tumor_Sample_Barcode'].value_counts())
    listOfDicts = []
    cntr = 0
    for case, cancerType in cancerTypeDict.items():
        if case in set(maf['Tumor_Sample_Barcode']) and case in set(expectedDf['case']):
        
            if cntr%50 == 0: print cntr,
            caseMaf = oncSnpMaf[oncSnpMaf['Tumor_Sample_Barcode'] == case]
            caseExpectation = expectedDf[expectedDf['case'] == case]
            relatedGenes = relatedGenesDict[cancerType]

            relatedGenesMaf = caseMaf[caseMaf['Hugo_Symbol'].isin(relatedGenes)]
            notRelatedGenesMaf = caseMaf[~caseMaf['Hugo_Symbol'].isin(relatedGenes)]
            relatedGenesExpected = caseExpectation[caseExpectation['gene'].isin(relatedGenes)]
            notRelatedGenesExpected = caseExpectation[~caseExpectation['gene'].isin(relatedGenes)]

            listOfDicts.append({'cancerType': cancerType, 'Tumor_Sample_Barcode': case,
                                'nmut_IM341': mutCounts[case],
                'nRelatedObserved': relatedGenesMaf.shape[0],
                'nUnrelatedObserved': notRelatedGenesMaf.shape[0],
                'nRelatedExpected': mutCounts[case]*sum(relatedGenesExpected['oncogenicChance']),
                'nUnrelatedExpected': mutCounts[case]*sum(notRelatedGenesExpected['oncogenicChance'])
            })

            cntr += 1
        
    df = pd.DataFrame(listOfDicts)
    return df

In [4]:
expectedDf = pd.read_table(filePathDict['EXPECTED_MUTATION_INFO_BY_GENE'])
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
relatedGenesDict = get_gene_and_cohort_list_utils.get_related_genes_by_cancer_type()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,17,18,40,41,43,44,71,108,124,125,128,129,130,131,132,133,134,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:302: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  allImpactMutsMaf = pd.read_table(impactMafPath) #todo get this in a better way
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py

In [7]:
cancerTypeDict = dict(get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath = filePathDict['CANCER_TYPE_INFO']))
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
hypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalIds = get_gene_and_cohort_list_utils.get_all_normal_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
allImpactMutsMaf['hypermutationStatus'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x:
    'hypermutated' if x in hypermutantIds else 'normal' if x in normalIds else 'Intermediate')
hypermutantMaf = allImpactMutsMaf[allImpactMutsMaf['hypermutationStatus'] == 'hypermutated']

In [10]:
df = summarize_obs_vs_expected_by_gene_type(hypermutantMaf, relatedGenesDict, expectedDf, cancerTypeDict)

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450


In [27]:
#dominantSignatureDict = get_gene_and_cohort_list_utils.get_pan_impact_signature_mapping(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'])
df['dominantSignature'] = df['Tumor_Sample_Barcode'].apply(lambda x:
        dominantSignatureDict[x] if x in dominantSignatureDict else None)
df['dominantSignature'] = df['dominantSignature'].apply(lambda x: 
        'APOBEC' if x == 'mean_APOBEC'
        else 'TMZ' if x == 'mean_11'
        else 'MMR' if x in ['mean_1', 'mean_MMR']
        #else 'UV' if x == 'mean_7'
        else 'POLE' if x == 'mean_10'
        #else 'SMOKING' if x == 'mean_SMOKING'
        else 'other')

In [28]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/obsExpectedRelatedGenesSup.tsv', index=False, sep='\t')

## Supplementary figure: enrichment of mutations in hypermutated tumors

In [ ]:
#samples a normal maf to have mutations drawn from similar distribution as hypermutated cases
def sample_normal_maf(normalMaf, hyperMaf, N=25):
    
    hyperCancerTypeCounter = Counter(hyperMaf['cancerType'])
    normalCancerTypeCounter = Counter(normalMaf['cancerType'])
    
    normalMaf['weight'] = normalMaf['cancerType'].apply(lambda x: 
        (1.0*normalMaf.shape[0]*hyperCancerTypeCounter[x])/(hyperMaf.shape[0]*normalCancerTypeCounter[x]))
    sampledCounts = normalMaf.sample(frac=N, weights='weight', replace=True)
    return sampledCounts

def summarize_mutational_prevalence(hyperMaf, normalMafSampled):
    
    nCasesHyper = len(set(hyperMaf['Tumor_Sample_Barcode']))
    nCasesNormal = len(set(normalMaf['Tumor_Sample_Barcode']))
    
    oncoMafNormal = normalMafSampled[normalMafSampled['oncogenic'].notnull()]
    oncoMafHyper = hyperMaf[hyperMaf['oncogenic'].notnull()]
    
    listOfDicts = []
    
    cntr = 0
    for gene in set(oncoMafNormal['Hugo_Symbol']) | set(oncoMafHyper['Hugo_Symbol']):
        
        geneMafNormal = oncoMafNormal[oncoMafNormal['Hugo_Symbol'] == gene]
        geneMafHyper = oncoMafHyper[oncoMafHyper['Hugo_Symbol'] == gene]
        normalCount = len(set(geneMafNormal['Tumor_Sample_Barcode']))
        hyperCount = len(set(geneMafHyper['Tumor_Sample_Barcode']))
        
        listOfDicts.append({'gene': gene,
            'countNormal': normalCount,
            'countHyper': hyperCount
        })
        
        cntr += 1
        if cntr%50 == 0: print cntr
        
    df = pd.DataFrame(listOfDicts)
    df['fracNormal'] = df['countNormal'].apply(lambda x: (1.0*x)/nCasesNormal)
    df['fracHyper'] = df['countHyper'].apply(lambda x:  (1.0*x)/nCasesHyper)
    
    return df
        

In [ ]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath=filePathDict['CANCER_TYPE_INFO'])
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)

msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = filePathDict['CASE_TMB_AND_MSI_STATS'], msiScoreThresh=10)
apobecCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_APOBEC')
poleCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_10')

hypermutatedIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalIds = get_gene_and_cohort_list_utils.get_all_normal_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
normalMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(normalIds)]


In [ ]:
listOfDfs = []
caseMapping = {'POLE': poleCases, 'APOBEC': apobecCases, 'MSI': msiCases}
"""for sigType, cases in caseMapping.items():
    print 'sampling', sigType
    hypermutatedMaf = allImpactMutsMaf[(allImpactMutsMaf['Tumor_Sample_Barcode'].isin(cases))]
    samples = sample_normal_maf(normalMaf, hypermutatedMaf, N=15)
    print 'summarizing'
    df = summarize_mutational_prevalence(hypermutatedMaf, samples)
    df['signature'] = sigType
    listOfDfs.append(df)
"""

#combinedDf = pd.concat(listOfDfs)
combinedDf['ratio'] = combinedDf['fracHyper']/combinedDf['fracNormal']
combinedDf['ddrGene'] = combinedDf['gene'].apply(lambda x: True if x in ['POLE', 'MLH1', 'MSH2', 'MSH6', 'PMS2'] else False)


In [ ]:
combinedDf.to_csv('~/Desktop/WORK/dataForLocalPlotting/percentComps.tsv', index=False, sep='\t')

## DNDS supplemental figure

In [ ]:
df = pd.read_table('/Users/friedman/Desktop/hypermutationProjectFinal/tables/table5.tsv')
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])

In [ ]:
relatedGenesDict = get_gene_and_cohort_list_utils.get_related_genes_by_cancer_type()

In [ ]:
#relatedGenesDict
df['related'] = df.apply(lambda row:
                'related' if row['CANCER_TYPE'] in relatedGenesDict and row['GENE'] in relatedGenesDict[row['CANCER_TYPE']]
                           else 'not-related', axis=1)
df['dndsIsSignificantHyper'] = df['DNDS_HYPER_SCORE'].apply(lambda x: 1 if x <.05 else 0)
df['cancerTypeGene'] = df.apply(lambda row: row['CANCER_TYPE'] + '_' + row['related'], axis=1)

In [ ]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/dndsSupplement.tsv',index=False, sep='\t')